In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\Corinne Vogel\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Corinne Vogel\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [13]:
#read data
factorDF = pd.read_csv("Europe_3_Factors.csv")
#drop all rows until 1999
print(factorDF.iloc[[102]]) #where date = 199901
x = range(0,114)
factorDF.drop(x, inplace=True)
#check that the right data is inplace (01.2000 as start date)
print(factorDF.head())
print(factorDF.tail())

#reset the index to 0-n
factorDF.reset_index(inplace=True, drop=True)
print(factorDF.head())

       Date  Mkt-RF   SMB   HML    RF
102  199901   -0.43 -0.16 -2.07  0.35
       Date  Mkt-RF   SMB   HML    RF
114  200001   -7.12  9.39 -6.12  0.41
115  200002    5.87  1.73 -9.50  0.43
116  200003    0.66 -5.25  3.23  0.47
117  200004   -5.58 -5.42  5.80  0.46
118  200005   -0.07 -0.12  6.06  0.50
       Date  Mkt-RF   SMB   HML    RF
381  202204   -5.86 -1.34  2.64  0.00
382  202205    1.27 -1.57  5.23  0.03
383  202206  -10.32 -1.05 -2.46  0.06
384  202207    5.26  0.62 -6.01  0.08
385  202208   -6.68 -0.72  5.41  0.19
     Date  Mkt-RF   SMB   HML    RF
0  200001   -7.12  9.39 -6.12  0.41
1  200002    5.87  1.73 -9.50  0.43
2  200003    0.66 -5.25  3.23  0.47
3  200004   -5.58 -5.42  5.80  0.46
4  200005   -0.07 -0.12  6.06  0.50


In [12]:
#we now have data from 200001 - 202209 (same as the profits from the momentum strategy)
#next up we regress the excess returns of our momentum strategy against the betas extracted from French's website

#first import the momentum and benchmark returns
momBenchReturn = pd.read_csv("5_11_Return_bench_mom.csv")

#check the first and last rows of the momBenchReturn dataframe
print(momBenchReturn.head())
print(momBenchReturn.tail())


#check whether they have the same length, i.e., same number of elements
print(len(momBenchReturn.mom_ret) == len(momBenchReturn.bench_ret))
    #outputs True

         Date  bench_ret   mom_ret  bench_indexed  mom_indexed
0  2000-01-31  -0.049517  0.051075       0.950483     1.051075
1  2000-02-29   0.067994  0.057946       1.067994     1.111980
2  2000-03-31   0.029254 -0.052601       1.099237     1.053489
3  2000-04-30   0.002649 -0.101531       1.102148     0.946528
4  2000-05-31  -0.026562 -0.064189       1.072873     0.885771
           Date  bench_ret   mom_ret  bench_indexed  mom_indexed
268  2022-05-31  -0.014752  0.033120       1.166837     1.777977
269  2022-06-30  -0.078505  0.027788       1.075234     1.827384
270  2022-07-31   0.075240 -0.080138       1.156134     1.680941
271  2022-08-31  -0.051596  0.043769       1.096482     1.754514
272  2022-09-30  -0.063805  0.022544       1.026521     1.794067
True


In [15]:
#create excess returns in a new dataframe
excess_return = {"Date": [], "mom-benchmark": [], "momentum_only": []}
for i in range(0, len(momBenchReturn.mom_ret)-1):
    excess_return["Date"].append(momBenchReturn.Date[i])
    excess_return["mom-benchmark"].append(momBenchReturn.mom_ret[i]-momBenchReturn.bench_ret[i])
    excess_return["momentum_only"].append(momBenchReturn.mom_ret[i])
excess_return = pd.DataFrame(excess_return)
#check the first elements, seems as if it has worked
print(excess_return.head())
print(excess_return.tail())
#works like a charm until 2022-08 --> until then we have data form French

         Date  mom-benchmark  momentum_only
0  2000-01-31       0.100592       0.051075
1  2000-02-29      -0.010048       0.057946
2  2000-03-31      -0.081854      -0.052601
3  2000-04-30      -0.104180      -0.101531
4  2000-05-31      -0.037627      -0.064189
           Date  mom-benchmark  momentum_only
267  2022-04-30       0.050410       0.039565
268  2022-05-31       0.047872       0.033120
269  2022-06-30       0.106294       0.027788
270  2022-07-31      -0.155378      -0.080138
271  2022-08-31       0.095365       0.043769


In [16]:
#create a new DF with the excess return, the non-excess return and the factors inside
regDF = {"excessReturn": excess_return["mom-benchmark"], "momReturn": excess_return["momentum_only"], "MktRf": factorDF["Mkt-RF"], "SMB": factorDF["SMB"], 
"HML": factorDF["HML"], "RF": factorDF["RF"]}

In [18]:
#now create the linear regression of these excess returns on the beta from the fama french
regression = smf.ols('excessReturn ~ MktRf + SMB + HML + RF', data=regDF).fit()
print(regression.summary())
#the momentum strategy cannot be explained by most of the factors excluding the market factor which can
#is significant

                            OLS Regression Results                            
Dep. Variable:           excessReturn   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     84.66
Date:                Thu, 10 Nov 2022   Prob (F-statistic):           2.48e-46
Time:                        09:52:45   Log-Likelihood:                 471.38
No. Observations:                 272   AIC:                            -932.8
Df Residuals:                     267   BIC:                            -914.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0033      0.003      0.969      0.3

In [22]:
regression2 = smf.ols('excessReturn ~ RF', data=regDF).fit()
print(regression2.summary())
#if we look at the AIC we see that we have a better fit if we only take the MktRf to regress on excess return
#log likelihood tiny bit less, but loss function is much higher in above model

                            OLS Regression Results                            
Dep. Variable:           excessReturn   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7177
Date:                Thu, 10 Nov 2022   Prob (F-statistic):              0.398
Time:                        09:56:40   Log-Likelihood:                 360.36
No. Observations:                 272   AIC:                            -716.7
Df Residuals:                     270   BIC:                            -709.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0012      0.005     -0.231      0.8

In [21]:
#now create the linear regression of these non-excess returns on the beta from the fama french
regression = smf.ols('momReturn ~ MktRf + SMB + HML + RF', data=regDF).fit()
print(regression.summary())
#the momentum strategy cannot be explained by most of the factors excluding the market factor which can
#is significant

                            OLS Regression Results                            
Dep. Variable:              momReturn   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     9.234
Date:                Thu, 10 Nov 2022   Prob (F-statistic):           5.30e-07
Time:                        09:53:23   Log-Likelihood:                 545.38
No. Observations:                 272   AIC:                            -1081.
Df Residuals:                     267   BIC:                            -1063.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0026      0.003      0.990      0.3